## Dyalog recipes


### Regular expressions

Dyalog supports the full PCRE syntax via ⎕S and ⎕R

See http://archive.vector.org.uk/art10500870

In [6]:
⍝ Get all words. The config-string '&' returns the matched string.
'\w+' ⎕S '&' ⊣ 'the cat sat on the mat'

┌───┬───┬───┬──┬───┬───┐
│the│cat│sat│on│the│mat│
└───┴───┴───┴──┴───┴───┘

In [7]:
⍝ Reverse each word
'\w+' ⎕S {⌽⍵.Match} ⊣ 'the cat sat on the mat'

┌───┬───┬───┬──┬───┬───┐
│eht│tac│tas│no│eht│tam│
└───┴───┴───┴──┴───┴───┘

Note that ⎕S is a dyadic function that returns a function. Left arg is always the pattern. Right arg is either a transformation function or a config string.

In [8]:
⍝ Capture groups
'c([^t]+)t' ⎕S '\1' ⊣ 'The cabriolet parked on the road'

┌───────┐
│abriole│
└───────┘

The config string is used to specify the capture group following perl conventions, \1, \2 etc

*Example*: given a vector of strings ABC)DE FDV)YST etc, make into two vectors:

a ← ABC FDV

b ← DE YST

etc

In [19]:
data ← 'A)B' 'B)C' 'C)D' 'D)E' 'E)F' 'B)G' 'G)H' 'D)I' 'E)J' 'J)K' 'K)L'
a b ← ↓⍉↑ '\w+' ⎕S '&' ¨ data
a b

┌───────────────────────┬───────────────────────┐
│┌─┬─┬─┬─┬─┬─┬─┬─┬─┬─┬─┐│┌─┬─┬─┬─┬─┬─┬─┬─┬─┬─┬─┐│
││A│B│C│D│E│B│G│D│E│J│K│││B│C│D│E│F│G│H│I│J│K│L││
│└─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┘│└─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┘│
└───────────────────────┴───────────────────────┘

The ⎕R function does a replace:

In [35]:
'[si]'⎕R'S' ⊢ 'mississippi'

mSSSSSSSppS

In [37]:
's'⎕R'S'⍠'ML' 2 ⊢ 'mississippi' ⍝ Set the "match limit" to 2, with the variant operator ⍠

miSSissippi

## Examples

FizzBuzz, as given in https://rosettacode.org/wiki/FizzBuzz#APL

In [8]:
⎕IO←0 ⍝ zero-based indexing
{⍵ 'Fizz' 'Buzz' 'FizzBuzz'[+/1 2×0=3 5|⍵]}¨1+⍳20

┌─┬─┬──────┬─┬──────┬──────┬─┬─┬──────┬──────┬──┬──────┬──┬──┬──────────┬──┬──┬──────┬──┬──────┐
│1│2│┌────┐│4│┌────┐│┌────┐│7│8│┌────┐│┌────┐│11│┌────┐│13│14│┌────────┐│16│17│┌────┐│19│┌────┐│
│ │ ││Fizz││ ││Buzz│││Fizz││ │ ││Fizz│││Buzz││ ││Fizz││ │ ││FizzBuzz││ │ ││Fizz││ ││Buzz││
│ │ │└────┘│ │└────┘│└────┘│ │ │└────┘│└────┘│ │└────┘│ │ │└────────┘│ │ │└────┘│ │└────┘│
└─┴─┴──────┴─┴──────┴──────┴─┴─┴──────┴──────┴──┴──────┴──┴──┴──────────┴──┴──┴──────┴──┴──────┘

The indexing function +/1 2×0=3 5|⍵ generates 0 for numbers not divisible by either 3 or 5, 1 for divisible by 3, 2 for divisible by 5 and 3 for divisible by 3 and 5. So how does that work?

Consider it right to left:

In [9]:
{3 5|⍵}¨10 11 15 ⍝ Return length 2 vector with remainder from div by 3 and 5

┌───┬───┬───┐
│1 0│2 1│0 0│
└───┴───┴───┘

Next step is to pick out the zeros

In [12]:
{0= 3 5|⍵}¨10 11 15 ⍝ Return length 2 vector with remainder from div by 3 and 5, highlighing zeros

┌───┬───┬───┐
│0 1│0 0│1 1│
└───┴───┴───┘

...and multiply by 1 and 2 respectively:

In [13]:
{1 2× 0= 3 5| ⍵}¨10 11 15 ⍝ ...multiplied by 1 and 2

┌───┬───┬───┐
│0 2│0 0│1 2│
└───┴───┴───┘

Now what remains is to sum that up:

In [14]:
{+/1 2× 0= 3 5|⍵}¨10 11 15 ⍝ ...multiplied by 1 and 2, and summed

2 0 3

We can now index into the vector containing our output text:

In [16]:
{⍵ 'Fizz' 'Buzz' 'FizzBuzz'[+/ 1 2× 0= 3 5| ⍵]} ¨ 3 10 11 15

┌──────┬──────┬──┬──────────┐
│┌────┐│┌────┐│11│┌────────┐│
││Fizz│││Buzz││ ││FizzBuzz││
│└────┘│└────┘│ │└────────┘│
└──────┴──────┴──┴──────────┘

### Split a string into a vector of a single char and a number

E.g 'X1234' gives 'X' 1234 -- via @Adám on APLOrchard

In [65]:
⎕IO←0 ⍝ zero-based indexing

In [66]:
1(↑,∘⍎↓)'X1234'

X 1234

Of course, using ⍎ on unfiltered user input can be dangerous. 

In [70]:
(⊃,∘⍎∩∘⎕D)'X4234' ⍝ safer, but can throw an error.

X 4234

The industrial strength version.

In [69]:
1(↑,1⊃∘⎕VFI↓)'X1234' 

X 1234

We can see how the first version is derived from a perhaps more obvious starting point:

In [71]:
{a←1↑⍵ ⋄ b←⍎1↓⍵ ⋄ a,b} 'X1234' ⍝ Take head, eval tail

{(1↑⍵),(⍎1↓⍵)} 'X1234'         ⍝ substitute variables
1 {(⍺↑⍵),(⍎⍺↓⍵)} 'X1234'       ⍝ break out 1
1 ((⊣↑⊢),(⍎⊣↓⊢)) 'X1234'       ⍝ train
1 ((↑),(⍎↓)) 'X1234'           ⍝ simplify
1 (↑,∘⍎↓) 'X1234'              ⍝ remove parens

X 1234

X 1234

X 1234

X 1234

X 1234

X 1234

### Partition a vector into groups of equal elements

In [33]:
⎕IO←0 ⍝ zero-based indexing
partition←{⍵⊂⍨1,2≠/⍵}
partition 1 1 1 1 1 2 2 1 4 4 4 4 1 1 2 2

┌─────────┬───┬─┬───────┬───┬───┐
│1 1 1 1 1│2 2│1│4 4 4 4│1 1│2 2│
└─────────┴───┴─┴───────┴───┴───┘

If we need the run lengths, we can just count them up:

In [34]:
≢¨partition 1 1 1 1 1 2 2 1 4 4 4 4 1 1 2 2

5 2 1 4 2 2

An alternative formulation, perhaps more performant is

In [30]:
{¯2-/¯1,⍸1,⍨2≠/⍵} 1 1 1 1 1 2 2 1 4 4 4 4 1 1 2 2

5 2 1 4 2 2

### Error handling

Using error guards

In [20]:
4 {3::'out of bounds' ⋄ ⍺ ⌷ ⍵} 1 2 3 4 5 6 7

5

In [21]:
7 {3::'out of bounds' ⋄ ⍺ ⌷ ⍵} 1 2 3 4 5 6 7

out of bounds

### Sets

Set XOR - union minus intersection, using a train

In [18]:
(∪~∩)/ (1 2 3 4 5 6) (4 5 6 7 8) ⍝ Elements in ⍺ or ⍵ but not in both

┌─────────┐
│1 2 3 7 8│
└─────────┘

### Encode and decode ⊤ ⊥

Encode and decode converts between decimal and some encoding vector.

Split a number into digits and pad with zeros from the left:

In [25]:
10 10 10 10 10 ⊤ 12

0 0 0 1 2

Convert from seconds to H M S:

In [34]:
0 24 60 60 ⊤ 8473

0 2 21 13

Decode goes the other way

In [36]:
0 24 60 60 ⊥ 0 2 21 13 

8473

Sometimes swapped to save some brackets

In [30]:
12 ⊤⍨ 5⍴10 ⍝ instead of (5⍴10) ⊤ 12

0 0 0 1 2

## Trains

Trains are APL's functional composition mechanism.

In [2]:
⍝ Construct a vector of the sum and product of two numbers
2 (+,×) 5

7 10

This is a three-function train, consisting of sum, catenate, product. For a three-function train (LMR) in the dyadic case, the execution flow is (⍺ L ⍵) M (⍺ R ⍵) or specifically (2+5),(2×5).

Here's another example: split a string on a separator:

In [20]:
',' (≠⊆⊢) 'one,two,three'

┌───┬───┬─────┐
│one│two│three│
└───┴───┴─────┘

Given the execution flow above we have:

In [8]:
Lhs ← ','≠'one,two,three'  ⍝ Binary match vector
Rhs ← ','⊢'one,two,three'  ⍝ Just return the right-hand argument
Lhs ⊆ Rhs                  ⍝ Partition based on binary match vector

┌───┬───┬─────┐
│one│two│three│
└───┴───┴─────┘

The right-tack ⊢ is used in trains as an identity function to force the correct monadic/dyadic behaviour.

Another canonical example is calculating the mean with a three-function monadic train:

In [9]:
(+/÷≢) 3 6 4 9

5.5

This translates to:

In [10]:
Lhs ← +/ 3 6 4 9   ⍝ Sum 
Rhs ← ≢ 3 6 4 9    ⍝ Count
Lhs ÷ Rhs          ⍝ Mean

5.5

Set XOR using a union-not-intersect train

In [19]:
(∪~∩)/ (1 2 3 4 5 6) (4 5 6 7 8)

┌─────────┐
│1 2 3 7 8│
└─────────┘

Here's a longer example: generate an integer range

In [100]:
5 (⊣+∘⍳-⍨) 10

5 6 7 8 9

## Tack tricks

Left (⊣) and right (⊢) tacks return the argument they point to.

In [12]:
'Left'⊣'Right' ⋄ 'Left'⊢'Right'

Left
Right

We saw earlier their place in trains to force dyadic application. Another use is when we want to return a value after first mutating it.

In [14]:
mem ← 1 2 3 4 5
mem ⊣ mem[1 2] ← 10 10

10 10 3 4 5

## Indexing

There are several ways of indexing into arrays and vectors. Crucially, elements of vectors and matrices are always scalars, but a scalar can be a boxed-up vector or matrix.

Indexing with [] or ⌷ returns the box, not the element, although if the element is a simple scalar, it's the same thing.

In [109]:
⎕IO←0 ⍝ zero-based indexing
v←⍳10
v[5]
v[5 2]
3⌷v

5

┌→──┐
│5 2│
└~──┘

3

In [111]:
v←(1 2 3)(4 5 6)(7 8 9)
v[0]  ⍝ Return the box at element 0

┌─────────┐
│ ┌→────┐ │
│ │1 2 3│ │
│ └~────┘ │
└∊────────┘

To get to the boxed element, we need to either disclose, or pick:

In [113]:
⊃v[0]  ⍝ Open box (disclose)
0⊃v    ⍝ Pick element at 0

┌→────┐
│1 2 3│
└~────┘

┌→────┐
│1 2 3│
└~────┘

The same box-unbox rules also apply to mutation:

In [118]:
v[1]←1 2 3 ⍝ Will fail with LENGTH ERROR, as value isn't boxed.

LENGTH ERROR
      v[1]←1 2 3 ⍝ Will fail with LENGTH ERROR, as value isn't boxed.
          ∧


Instead we need to explicitly enclose the new vector:

In [117]:
v[1]←⊂1 2 3
v

┌→────────────────────────┐
│ ┌→────┐ ┌→────┐ ┌→────┐ │
│ │1 2 3│ │1 2 3│ │7 8 9│ │
│ └~────┘ └~────┘ └~────┘ │
└∊────────────────────────┘

Set multiple values with @ without mutation

In [46]:
9 9 @ 4 5 ⊢ v             ⍝ Note -- no mutation
v 

0 1 2 3 9 9 6 7 8 9

0 1 2 3 8 8 6 7 8 9

In [40]:
v ⊣ v[4 5] ← 8 8          ⍝ Note -- direct assignment mutates v

0 1 2 3 8 8 6 7 8 9

The @ operator can also apply functions (via @Adám on APLOrchard):

In [4]:
(-@2 5)10 20 30 40 50 60  ⍝ Apply monadic - at positions 2 and 5

10 ¯20 30 40 ¯50 60

which also works for dyads:

In [5]:
7(+@2 5)10 20 30 40 50 60 

10 27 30 40 57 60

In [10]:
'x'@(∊∘⎕A)'Hello World' ⍝ Replace all uppercase letters with 'x'. ⎕A gives the uppercase letters.

xello xorld

## Products

Cartesian product

In [17]:
⎕IO←0
pairs ← {,(⍳⍵)∘.,⍳⍵}  ⍝ Python [[x, y] for x in range(3) for y in range(3)]
pairs 3

┌───┬───┬───┬───┬───┬───┬───┬───┬───┐
│0 0│0 1│0 2│1 0│1 1│1 2│2 0│2 1│2 2│
└───┴───┴───┴───┴───┴───┴───┴───┴───┘

or an alternative formulation, using iota. Think of the arguments to iota as defining the shape of the result:

In [18]:
pairs ← {,⍳⍵ ⍵}
pairs 3

┌───┬───┬───┬───┬───┬───┬───┬───┬───┐
│0 0│0 1│0 2│1 0│1 1│1 2│2 0│2 1│2 2│
└───┴───┴───┴───┴───┴───┴───┴───┴───┘

## Composition (Currying)

We can curry a dyadic function down to a monadic function by fixing either left or right argument:

In [55]:
sum←{⍺+⍵}
add5←5∘sum

In [57]:
add5 7

12

In [58]:
add5 1 2 3 4 5 6 7

6 7 8 9 10 11 12

## Shape and Rank

Pair consecutive elements in a vector

In [80]:
{⊂⍵}⌺(⍪2 2) ⊢ ⍳10

┌───┬───┬───┬───┬───┐
│0 1│2 3│4 5│6 7│8 9│
└───┴───┴───┴───┴───┘

In [76]:
,⌿⍉5 2 ⍴ ⍳10

┌───┬───┬───┬───┬───┐
│0 1│2 3│4 5│6 7│8 9│
└───┴───┴───┴───┴───┘

In [77]:
↓5 2 ⍴ ⍳10

┌───┬───┬───┬───┬───┐
│0 1│2 3│4 5│6 7│8 9│
└───┴───┴───┴───┴───┘

## Folds

The reduce operator / folds R-L. It's possible to define a fold operator that works L-R:

In [90]:
foldl←{↑⍺⍺⍨/(⌽⍵),⊂⍺}

In [92]:
0 + foldl 1 2 3 4 5 ⍝ Must give initial accumulator state

15

Note that this operator (and many others) exists in the standard Dyalog workspace [dfns](https://dfns.dyalog.com/s_foldl.htm), which can be imported as ⎕CY 'dfns'

## On ranges

In Python you can specify a range as `range(start, end)` which gives an iterator from start to end-1. In Dyalog we don't have a direct equivalent, but one can be made from the iota operator.

In [93]:
5 {⍺↓⍳⍵} 10   ⍝ Very wasteful!

5 6 7 8 9

In [94]:
5 {⍺+⍳⍵-⍺} 10 ⍝ Better

5 6 7 8 9

In [98]:
5 (⊣+∘⍳-⍨) 10 ⍝ As a train

5 6 7 8 9

## Stencils

Pick out 3x3 regions of a larger matrix. The stencil takes a function to the left and a shape to the right and returns a monad.

This from https://chat.stackexchange.com/rooms/52405/conversation/lesson-5-even-more-apl-operators--

In [21]:
⎕←letters ← 4 6⍴⎕A
{⊂⍵}⌺3 3 ⊢ letters     ⍝ Enclose the selected window

ABCDEF
GHIJKL
MNOPQR
STUVWX

┌───┬───┬───┬───┬───┬───┐
│ │ │ │ │ │ │
│ AB│ABC│BCD│CDE│DEF│EF │
│ GH│GHI│HIJ│IJK│JKL│KL │
├───┼───┼───┼───┼───┼───┤
│ AB│ABC│BCD│CDE│DEF│EF │
│ GH│GHI│HIJ│IJK│JKL│KL │
│ MN│MNO│NOP│OPQ│PQR│QR │
├───┼───┼───┼───┼───┼───┤
│ GH│GHI│HIJ│IJK│JKL│KL │
│ MN│MNO│NOP│OPQ│PQR│QR │
│ ST│STU│TUV│UVW│VWX│WX │
├───┼───┼───┼───┼───┼───┤
│ MN│MNO│NOP│OPQ│PQR│QR │
│ ST│STU│TUV│UVW│VWX│WX │
│ │ │ │ │ │ │
└───┴───┴───┴───┴───┴───┘

The padding is defined by ⍺ in the operand function:

In [22]:
({⊂⍺}⌺3 3) ⊢ letters

┌────┬────┬────┬────┬────┬─────┐
│1 1 │1 0 │1 0 │1 0 │1 0 │1 ¯1 │
├────┼────┼────┼────┼────┼─────┤
│0 1 │0 0 │0 0 │0 0 │0 0 │0 ¯1 │
├────┼────┼────┼────┼────┼─────┤
│0 1 │0 0 │0 0 │0 0 │0 0 │0 ¯1 │
├────┼────┼────┼────┼────┼─────┤
│¯1 1│¯1 0│¯1 0│¯1 0│¯1 0│¯1 ¯1│
└────┴────┴────┴────┴────┴─────┘

In [23]:
({⊂⍺↓⍵}⌺3 3) ⊢ letters

┌──┬───┬───┬───┬───┬──┐
│AB│ABC│BCD│CDE│DEF│EF│
│GH│GHI│HIJ│IJK│JKL│KL│
├──┼───┼───┼───┼───┼──┤
│AB│ABC│BCD│CDE│DEF│EF│
│GH│GHI│HIJ│IJK│JKL│KL│
│MN│MNO│NOP│OPQ│PQR│QR│
├──┼───┼───┼───┼───┼──┤
│GH│GHI│HIJ│IJK│JKL│KL│
│MN│MNO│NOP│OPQ│PQR│QR│
│ST│STU│TUV│UVW│VWX│WX│
├──┼───┼───┼───┼───┼──┤
│MN│MNO│NOP│OPQ│PQR│QR│
│ST│STU│TUV│UVW│VWX│WX│
└──┴───┴───┴───┴───┴──┘

## Key

In [24]:
{⍺,≢⍵}⌸'Mississippi' 

M 1
i 4
s 4
p 2

## Sliding Tiles example

In [39]:
⎕IO←0 ⍝ zero-based indexing
]box on -style=max ⍝ Pass all output through DISPLAY

┌→────────────────┐
│Was ON -style=max│
└─────────────────┘

In [40]:
]dinput
moves←{
     d←(0 1)(1 0)(0 ¯1)(¯1 0)      ⍝ Move offsets
     m←⍵
     {⌽@i ⍵⊢m}¨(,⍳⍴m)∩d+⊂i←⊃⍸0=m
 }

In [41]:
state←4 4⍴?⍨16
moves state

┌→──────────────────────────────────────────┐
│ ┌→──────────┐ ┌→──────────┐ ┌→──────────┐ │
│ ↓ 0 10 3 2│ ↓15 10 3 2│ ↓15 10 3 2│ │
│ │15 5 13 9│ │ 5 0 13 9│ │12 5 13 9│ │
│ │12 4 8 14│ │12 4 8 14│ │ 0 4 8 14│ │
│ │11 7 6 1│ │11 7 6 1│ │11 7 6 1│ │
│ └~──────────┘ └~──────────┘ └~──────────┘ │
└∊──────────────────────────────────────────┘

A lot to decode here. The `i` expression gives the row-col of the 0 element:

In [42]:
state ⋄ ⊃⍸0=state

┌→──────────┐
↓15 10 3 2│
│ 0 5 13 9│
│12 4 8 14│
│11 7 6 1│
└~──────────┘
┌→──┐
│1 0│
└~──┘

Adding the offsets gives all potential moves:

In [43]:
d←(0 1)(1 0)(0 ¯1)(¯1 0)
d+⊂i←⊃⍸0=state                ⍝ disclose-enclose as we want the enclosed version later

┌→─────────────────────────┐
│ ┌→──┐ ┌→──┐ ┌→───┐ ┌→──┐ │
│ │1 1│ │2 0│ │1 ¯1│ │0 0│ │
│ └~──┘ └~──┘ └~───┘ └~──┘ │
└∊─────────────────────────┘

Given the rank of m we can generate all valid coordinate pairs

In [44]:
,⍳⍴state

┌→─────────────────────────────────────────────────────────────────────────────
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐
│ │0 0│ │0 1│ │0 2│ │0 3│ │1 0│ │1 1│ │1 2│ │1 3│ │2 0│ │2 1│ │2 2│ │2 3│ │3 0│
│ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘
└∊─────────────────────────────────────────────────────────────────────────────

 ───────────────────┐
 ┌→──┐ ┌→──┐ ┌→──┐ │
 │3 1│ │3 2│ │3 3│ │
 └~──┘ └~──┘ └~──┘ │
 ───────────────────┘

and then discard the moves that falls out of range as an intersection:

In [45]:
(,⍳⍴state)∩d+⊂i←⊃⍸0=state

┌→──────────────────┐
│ ┌→──┐ ┌→──┐ ┌→──┐ │
│ │0 0│ │1 1│ │2 0│ │
│ └~──┘ └~──┘ └~──┘ │
└∊──────────────────┘

We now map `{⌽@i ⍵⊢m}` over the list of valid moves. How does that work?

In [46]:
{⌽@i ⍵⊢state}¨(,⍳⍴state)∩d+⊂i←⊃⍸0=state

┌→──────────────────────────────────────────┐
│ ┌→──────────┐ ┌→──────────┐ ┌→──────────┐ │
│ ↓ 0 10 3 2│ ↓15 10 3 2│ ↓15 10 3 2│ │
│ │15 5 13 9│ │ 5 0 13 9│ │12 5 13 9│ │
│ │12 4 8 14│ │12 4 8 14│ │ 0 4 8 14│ │
│ │11 7 6 1│ │11 7 6 1│ │11 7 6 1│ │
│ └~──────────┘ └~──────────┘ └~──────────┘ │
└∊──────────────────────────────────────────┘

The dyadic `@` glyph in this case applies the reverse function `⌽` on the set of indexes given to its right argument. For example, if we want to switch the elements at (1 0) and (1 1) in m, we could say:

In [47]:
state ⋄ (⌽@(1 0) (1 1)) state

┌→──────────┐
↓15 10 3 2│
│ 0 5 13 9│
│12 4 8 14│
│11 7 6 1│
└~──────────┘
┌→──────────┐
↓15 10 3 2│
│ 5 0 13 9│
│12 4 8 14│
│11 7 6 1│
└~──────────┘

Using the right-tack `⊢` we can drop the brackets.

In [48]:
⌽@(1 0) (1 1) ⊢ state

┌→──────────┐
↓15 10 3 2│
│ 5 0 13 9│
│12 4 8 14│
│11 7 6 1│
└~──────────┘

In [49]:
]dinput
unwind←{
     prev←⍺
     ⌽⍬{
         row←prev[;0]⍳⊂⍵
         prev[row;0]≡prev[row;1]:⍺
         (⍺,⊂⍵)∇⊃prev[row;1]
     }⍵
 }

In [52]:
]dinput
solve←{                                ⍝ Breadth-first search.
     goal←2 2⍴1+¯1,⍨⍳3
     prev←⊃⍵ ⍬
     ⍬{
         ⍵≡⍬:⍬                          ⍝ Empty q, not found
         head←⊃⍵ ⋄ tail←1↓⍵             ⍝ Text and remaining states in queue.
         head≡goal:prev                 ⍝ Goal found
         next←moves head
         prev⍪←(next~⍺),[0.5]⊂head
         (⍺,⊂head)∇(tail∪next)~⍺        ⍝ accumulate visited states
     }⍵                                 ⍝ Start state
 }

In [38]:
solve ⊂state

LENGTH ERROR
solve[7] (⍺,head)∇(tail∪next)~⍺  ⍝ accumulate visited vertices.
           ∧


In [51]:
1↑⊂state

┌→──────────────┐
│ ┌→──────────┐ │
│ ↓15 10 3 2│ │
│ │ 0 5 13 9│ │
│ │12 4 8 14│ │
│ │11 7 6 1│ │
│ └~──────────┘ │
└∊──────────────┘

## A heap

A binomial heap is a tree-like data structure maintaining a partial ordering such that a parent node has a value guaranteed to be smaller than either of its children, but no ordering between the children is specified. 

The classic heap structure lends itself naturally to imperative implementations, and is not a natural fit for APL. We'll do better later!

To illustrate the point, here is a partial port of Python's heapq (https://github.com/python/cpython/blob/2.7/Lib/heapq.py)

In [86]:
⎕IO←0 ⍝ zero-based indexing
]box on -style=max ⍝ Pass all output through DISPLAY

┌→────────────────┐
│Was ON -style=max│
└─────────────────┘

In [96]:
]dinput
Push←{ 
    ⍝ Insert item ⍵ into heap ⍺, returning the resulting tree
    ⍝ Insert a new item by sticking it at the end and sifting the heap
    ⍝ to maintain the heap invariant.
    ⍺←⍬
    Siftdown (⍺,⍵) 0 (≢⍺)
}

In [97]:
]dinput
Pop←{ ⍝ Pop the smallest item off the heap, maintaining the heap invariant.
    heap←⍵
    last←⊃¯1↑heap ⋄ rest←¯1↓heap
    0=≢rest:rest last
    min←heap[0]
    newHeap←(last@0)rest
    (Siftup newHeap 0) min
}

In [98]:
]dinput
Siftdown←{
    ⍝ 'heap' is a heap at all indices >= startpos, except possibly for pos. Pos
    ⍝ is the index of a leaf with a possibly out-of-order value.  Restore the
    ⍝ heap invariant
    (heap start pos)←⍵
    item←pos⌷heap
    ⍝ Follow the path to the root, moving parents down until finding a place
    ⍝ newitem fits.
    newpos←{
        ⍵≤start:⍵
        parentpos←⌊(⍵-1)÷2
        parent←parentpos⌷heap
        item<parent:∇ parentpos⊣heap[⍵]←parent
        ⍵
    } pos
    (item@newpos)heap
}

In [99]:
]dinput
Siftup←{
    ⍝ Bubble the smaller child (and so on with that child's children, etc) 
    ⍝ until hitting a leaf, then using Siftdown to move the oddball originally 
    ⍝ at index current into place.
    (heap current)←⍵
    newItem←heap[current]
    leafPos←{ ⍝ Bubble up the smaller child until hitting a leaf.
        pos←⍵
        left←1+2×pos    ⍝ left child position
        left≥≢heap:pos
        right←1+left    ⍝ right child position
        child←((right<≢heap)∧~heap[left]<heap[right])⊃left right
        heap[pos]←heap[child]
        ∇ child
    } current
    ⍝ The leaf at leafPos is empty now.  Put newitem there, and bubble it up
    ⍝ to its final resting place (by sifting its parents down).
    heap[leafPos]←newItem
    Siftdown heap current leafPos
}

In [100]:
heap←0 1 2 5 6 8 9
Pop heap

┌→────────────────┐
│ ┌→──────────┐ │
│ │1 5 2 9 6 8│ 0 │
│ └~──────────┘ │
└∊────────────────┘

In [102]:
heap Push←3
heap

┌→──────────────┐
│0 1 2 3 6 8 9 5│
└~──────────────┘

## A heap implemented as a Leftist Tree

A heap can also be implemented as a Leftist Tree, which lends itself naturally to a recursive implementation.

https://en.wikipedia.org/wiki/Leftist_tree
http://typeocaml.com/2015/03/12/heap-leftist-tree/

Here is a partial port of the OCaml version.

In [104]:
]dinput
Pop←{ ⍝ Pop off smallest element from a leftist tree ⍵
    0=≢⍵:⍬
    (value left right)←1↓⍵
    (left Merge right) value
}

In [105]:
]dinput
Push←{ ⍝ Insert item ⍵ into leftist tree ⍺, returning the resulting tree
    ⍺←⍬              ⍝ default to init
    1 ⍵ ⍬ ⍬ Merge ⍺ 
}

In [119]:
]dinput
Merge←{ ⍝ Merge leftist trees ⍺ and ⍵
    0=≢⍺:⍵ ⋄ 0=≢⍵:⍺                                 ⍝ If either is a leaf, return the other
    (key left right)←1↓⍺
    key>1⊃⍵:⍵∇⍺                                     ⍝ Flip to ensure smallest is root of merged
    merged←right∇⍵                                  ⍝ Merge rightwards
    leftRank←⊃left ⋄ mergedRank←⊃merged
    leftRank≥⊃merged:(1+mergedRank) key left merged ⍝ Right is shorter
    (1+leftRank) key merged left                    ⍝ Left is shorter; make it the new right
}

In [120]:
⍝ Example given in http://typeocaml.com/2015/03/12/heap-leftist-tree/
h←Push 2
h Push←10
h Push←9

s←Push 3
s Push←6

h Merge s

┌→─────────────────────────────────────────────────────────────┐
│ ┌→────────────────────────────────────┐ ┌→─────────────┐ │
│ 2 2 │ ┌→────────────┐ ┌→────────────┐ │ │ ┌⊖┐ ┌⊖┐ │ │
│ │ 2 3 │ ┌⊖┐ ┌⊖┐ │ │ ┌⊖┐ ┌⊖┐ │ │ │ 1 10 │0│ │0│ │ │
│ │ │ 1 6 │0│ │0│ │ │ 1 9 │0│ │0│ │ │ │ └~┘ └~┘ │ │
│ │ │ └~┘ └~┘ │ │ └~┘ └~┘ │ │ └∊─────────────┘ │
│ │ └∊────────────┘ └∊────────────┘ │ │
│ └∊────────────────────────────────────┘ │
└∊─────────────────────────────────────────────────────────────┘

More compact, and a more natural fit for APL. Note that the Leftist Tree has slightly different performance characteristics to the standard heap. 